# P1: MLP Implementation

**Objective:** Implement a simple Multilayer Perceptron (MLP) using TensorFlow/Keras and report training/validation accuracy.

In [ ]:
# Minimal MLP example using Keras
import tensorflow as tf
from tensorflow.keras import layers, models
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28*28).astype('float32')/255.0
x_test = x_test.reshape(-1, 28*28).astype('float32')/255.0
model = models.Sequential([layers.Input(shape=(28*28,)), layers.Dense(128, activation='relu'), layers.Dropout(0.2), layers.Dense(10, activation='softmax')])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_split=0.1, epochs=3, batch_size=128)

**Summary:** This notebook demonstrates a compact MLP pipeline. Extend by adding regularization, batch normalization, and experiments with optimizers.

In [ ]:
# Practical 1: MLP on sklearn digits (PyTorch)
import numpy as np
import torch, torch.nn as nn, torch.optim as optim
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

# Load data
digits = load_digits()
X = digits.data.astype(np.float32)
y = digits.target.astype(np.int64)
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_t = torch.from_numpy(X_train)
y_train_t = torch.from_numpy(y_train)
X_test_t = torch.from_numpy(X_test)
y_test_t = torch.from_numpy(y_test)

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(64, 128), nn.ReLU(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 10) )
    def forward(self, x):
        return self.net(x)

model = MLP()
criterion = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=1e-3)

train_losses = []
for epoch in range(20):
    model.train()
    logits = model(X_train_t)
    loss = criterion(logits, y_train_t)
    opt.zero_grad(); loss.backward(); opt.step()
    train_losses.append(loss.item())

model.eval()
with torch.no_grad():
    pred = model(X_test_t).argmax(1).numpy()
acc = accuracy_score(y_test, pred)
print('Test accuracy:', acc)

plt.figure(figsize=(5,3))
plt.plot(train_losses); plt.title('Train Loss'); plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.show()

from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay(confusion_matrix(y_test, pred)).plot(values_format='d'); plt.show()